In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
#Importo el CSV para limpiarlo y transformarlo:
scrap_cuadros_completo = pd.read_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/scrap_cuadros_completo.csv")

In [3]:
dim_original = scrap_cuadros_completo.shape # De esta forma guardo las dimensiones del data frame original.
scrap_cuadros_completo.shape

(44724, 10)

In [4]:
cuadros_completo = scrap_cuadros_completo.copy() #Hago una copia del original por si luego quiero acceder a él.

# Exploración y transformación.

En primer lugar, puesto que nuestro objeto de estudio es acerca dela información de iconografía en las obras del Museo Del Prado, eliminaré todas aquellas filas en las que los datos de las cinco columnas pertenecientes a este grupo sean valores nulos.

In [5]:
columnas_interes = ['Fauna', 'Flora', 'Personajes', 'Objetos', 'Geografico']
cuadros_completo = cuadros_completo.dropna(subset=columnas_interes, how='all')

In [6]:
cuadros_completo.head()

,Titulo,Soporte_Fecha,Autor,Personajes,Objetos,Flora,Fauna,Geografico,Escuela,Ubicacion
0,El sumo sacerdote Aarón,Óleo sobre tabla. 1545 - 1550,"JUANES, JUAN DE",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 051\nEliminar
1,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
2,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Moisés\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
3,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Abacuc\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar
4,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Amós\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar


In [7]:
cuadros_completo.columns # Esto nos proporciona una enumeración de los nombres correspondientes a cada una de las columnas presentes en los datos.

Index(['Titulo', 'Soporte_Fecha', 'Autor', 'Personajes', 'Objetos', 'Flora',
       'Fauna', 'Geografico', 'Escuela', 'Ubicacion'],
      dtype='object')

Como en el scrapeo de los datos, la información acerca del soporte y la fecha la he obtenido de forma conjunta, voy a separar esta columna en dos diferentes, diferenciando la información específica de cada elemento.

In [8]:

cuadros_completo[['Soporte', 'Fecha']] = cuadros_completo['Soporte_Fecha'].str.split('.', n=1, expand=True)

In [9]:
cuadros_completo.drop('Soporte_Fecha', axis=1, inplace=True) #elimino esta columna porque la información de la misma está ya contenida en las dos recién creadas.

In [10]:
cuadros_completo.info() # De este modo conzco el tipo de dato y los valores nulos que tiene cada una de las columnas.

<class 'pandas.core.frame.DataFrame'>
Index: 42633 entries, 0 to 42632
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Titulo      42633 non-null  object
 1   Autor       42599 non-null  object
 2   Personajes  13157 non-null  object
 3   Objetos     30494 non-null  object
 4   Flora       27784 non-null  object
 5   Fauna       30674 non-null  object
 6   Geografico  4508 non-null   object
 7   Escuela     11756 non-null  object
 8   Ubicacion   31382 non-null  object
 9   Soporte     42633 non-null  object
 10  Fecha       42608 non-null  object
dtypes: object(11)
memory usage: 3.9+ MB


In [11]:
#Relleno aquí los valores nulos con la palabra "Desconocido":
cuadros_completo = cuadros_completo.fillna('Desconocido')

# 1. "Soporte".

In [12]:
#cuadros_completo['Soporte'].unique()

In [13]:
cuadros_completo.loc[cuadros_completo['Soporte'] == 'Después de 1874']

,Titulo,Autor,Personajes,Objetos,Flora,Fauna,Geografico,Escuela,Ubicacion,Soporte,Fecha
3365,Guerras carlistas de 1873-1874,ANÓNIMO,Alfonso XII\nEliminar,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Después de 1874,Desconocido


He podido observar que al separar la columna "Soporte_Fecha" existente anteriormente, como lo he hecho a partir del primer punto existente, al no contar, en algunos casos con la información del Soporte, la in formación de la Fecha se ha quedado en la columna de Soporte, por lo tanto, voy a modificar esto:

In [14]:
# Definir una función lambda para verificar si hay un número o la palabra "Siglo" en la cadena
condicion = lambda x: any(char.isdigit() for char in x) or 'Siglo' in x



In [15]:
# Filtrar las filas que cumplen con la condición
filas_condicion = cuadros_completo[cuadros_completo['Soporte'].apply(condicion)]

In [16]:
# Mover los valores de 'Soporte' a 'Fecha' y asignar el valor 'Desconocido' a 'Soporte'
cuadros_completo.loc[filas_condicion.index, 'Fecha'] = cuadros_completo.loc[filas_condicion.index, 'Soporte']
cuadros_completo.loc[filas_condicion.index, 'Soporte'] = 'Desconocido'


# 2. "Fecha"

In [17]:
cuadros_completo[['Fecha']].head()

,Fecha
0,1545 - 1550
1,1566 - 1568
2,1566 - 1568
3,1435 - 1440
4,1435 - 1440


In [18]:
def transformar_fechas(fecha):
    dic_fechas = {
        'Primer cuarto del siglo XVII': '1625',
        'Siglo XVII': '1650',
        'Siglos XVII': '1650',
        'Finales del siglo XVII - Principio del siglo XVIII': '1700',
        'Finales del siglo III a.C. - Primera mitad del siglo II a.C.': '(-225)',
        'Siglo XVI': '1550',
        'Segunda mitad del siglo XVI': '1575',
        'Principio del siglo XX': '1901',
        'Último tercio del siglo XVIII': '1790',
        'Finales del siglo XVI - Primer tercio del siglo XVII': '1600',
        'Primera mitad del siglo XVII': '1625',
        'Segunda mitad del siglo XVII': '1675',
        'Finales del siglo XVII': '1692',
        'Siglo XVIII': '1750',
        'Finales del siglo XVIII': '1792',
        'Siglos XVI - XVII': '1600',
        'Segunda mitad del siglo XVII - Primer cuarto del siglo XVIII': '1685',
        'Siglo I a.C.': '(-100)',
        'Finales del siglo XVI': '1692',
        'Primer tercio del siglo XVII': '1615',
        'Siglo III a.C.': '(-300)',
        'Primer tercio del siglo XVI': '1515',
        'Segundo tercio del siglo XVII': '1660',
        'Siglo XII': '1150',
        'Último cuarto del siglo XVIII - Principio del siglo XIX': '1792',
        'Último cuarto del siglo XVII': '1690',
        'Siglos IV a.C. - III a.C.': '(-300)',
        'Mediados del siglo XVII': '1650',
        'Siglo XIX': '1850',
        'XVI': '1550',
        'Primer cuarto del siglo XX': '1925',
        'Principio del siglo II a.C': '(-200)',
        '211 a.C': '(-211)',
        'Primera mitad del siglo II a.C': '(-200)',
        '212 a.C': '(-212)',
        '218 a.C': '(-218)',
        '241 a.C': '(-241)',
        'Finales del siglo XV': '1492',
        'Siglo XX': '1959',
        'Primer cuarto del siglo III':'200',
        'Siglo XV': '1450',
        '9 a.C': '(-9)',
        '27 a.C': '(-27)',
        'Último cuarto del siglo XV': '1475',
        'Primera mitad del siglo XX': '1925',
        'Mediados del siglo I': '50',
        'Siglo II a.C': '(-200)',
        'Mediados del siglo II a.C': '(-125)'
    }

    for clave, valor in dic_fechas.items():
        if clave.lower() in fecha.lower():
            return valor

    return fecha

cuadros_completo['Fecha'] = cuadros_completo['Fecha'].apply(transformar_fechas)


In [19]:
def eliminar_temp(fecha):
    palabras_eliminar = ['Hacia', 'Después de', 'Antes de']

    for palabra in palabras_eliminar:
        if palabra in fecha:
            fecha = fecha.replace(palabra, '').strip()

    return fecha
# Aplicar la función a toda la columna 'Fecha'
cuadros_completo['Fecha'] = cuadros_completo['Fecha'].apply(eliminar_temp)

In [20]:
filtro = ~cuadros_completo['Fecha'].str.contains(r'\d|X|V|I|Desconocido', regex=True)
#Vamos a rellenarlos con la palabra "Desconocido"
cuadros_completo.loc[filtro, 'Fecha'] = 'Desconocido'

In [21]:
# Eliminar '-…' al final de cada valor en la columna 'Fecha'
cuadros_completo['Fecha'] = cuadros_completo['Fecha'].str.rstrip('-...')

In [22]:
cuadros_completo['Fecha'].isnull().sum()

0

In [23]:
def obtener_segundo_numero(fecha):
    # Verificar si hay un guion ("-") en la cadena
    if '-' in fecha:
        partes = fecha.split('-')
        
        # Verificar si la segunda parte es un número o convertible a número
        try:
            segundo_numero = int(partes[1].strip())
        except ValueError:
            # En caso de que no sea un número o no sea convertible a número, asignar 'Desconocido'
            segundo_numero = 'Desconocido'
        
        return segundo_numero
    else:
        # Si no hay guion, conservar el valor original
        return fecha

cuadros_completo['Fecha'] = cuadros_completo['Fecha'].apply(obtener_segundo_numero)



# 3. "Autor".

In [24]:
cuadros_completo['Autor'].str.contains('Desconocido').any()

True

In [25]:
cuadros_completo['Autor'].str.contains('Desconocido').sum()

34

In [26]:
#uadros_completo['Autor'].unique()

# 4. Columnas relacionadas con la Iconografía "Escuela" y "Ubicación".

In [30]:
columnas_a_limpiar = ['Personajes', 'Objetos', 'Flora', 'Fauna', 'Geografico', 'Escuela', 'Ubicacion']

In [31]:
def reemplazar_coma(columna):
    if '\nEliminar\n' in columna:
        return columna.replace('\nEliminar\n', ', ')
    elif '\nEliminar' in columna:
        return columna.replace('\nEliminar', '')
    else:
        return columna

# Aplicar la función a las columnas especificadas
for columna in columnas_a_limpiar:
    cuadros_completo[columna] = cuadros_completo[columna].apply(reemplazar_coma)


In [33]:
objetos_eliminar = [
    "Armas", "Objeto de escritorio", "Alimentos", 
    "Objeto para la Higiene", "Objeto de Iluminación", 
    "Objetos de Iconografía Cristiana", "Reloj", 
    "Textil", "Objeto Uso Individual", 
    "Objetos Artísticos", "Objeto Óptico", 
    "Elemento de Arquitectura", "Enseres Domésticos", 
    "Instrumentos Musicales", "Libro", 
    "Elementos Heráldicos y Nobiliarios", 
    "Elementos para Comercio", 
    "Herramienta / Utensilios de Trabajo", 
    "Transporte", "Elementos Mitológicos", 
    "Instrumentos Científicos", 
    "Juego / Juguete", "Bandera", 
    "Mueble", "Espejo"
]

# Identificar las filas que cumplen con la condición
condicion_obj_eliminar = cuadros_completo['Objetos'].isin(objetos_eliminar)



In [34]:
# Eliminar las filas que cumplen con la condición
cuadros_completo.drop(cuadros_completo[condicion_obj_eliminar].index, inplace=True)

In [35]:
#cuadros_completo['Objetos'].unique()

In [36]:
geo_eliminar = [
    "España", "Mexico (Méjico)", "Alemania", 
    "Israel", "Holanda", "Francia", 
    "Italia", "Portugal", "Reino Unido", 
    "Suiza", "Turquía", "Antillas", 
    "Bélgica", "Magreb", "República Checa (Chequia)", 
    "Eslovenia", "Brasil", "Egipto", "Gracia"
]

# Identificar las filas que cumplen con la condición
condicion_eliminar_geo = cuadros_completo['Geografico'].isin(geo_eliminar)

# Eliminar las filas que cumplen con la condición
cuadros_completo.drop(cuadros_completo[condicion_eliminar_geo].index, inplace=True)


In [37]:
#cuadros_completo['Geografico'].unique()

In [39]:
cuadros_completo.isnull().sum()

Titulo        0
Autor         0
Personajes    0
Objetos       0
Flora         0
Fauna         0
Geografico    0
Escuela       0
Ubicacion     0
Soporte       0
Fecha         0
dtype: int64

In [40]:
cuadros_completo.tail()

,Titulo,Autor,Personajes,Objetos,Flora,Fauna,Geografico,Escuela,Ubicacion,Soporte,Fecha
42625,Incendio en la Cartuja de Praga y persecución ...,"CARDUCHO, VICENTE",Desconocido,Desconocido,Desconocido,Desconocido,"República Checa (Chequia), Praga",Desconocido,Desconocido,Óleo sobre lienzo,1632
42627,Incendio en la Cartuja de Praga y persecución ...,"CARDUCHO, VICENTE",Desconocido,Desconocido,Desconocido,Desconocido,"República Checa (Chequia), Praga",Desconocido,Desconocido,Óleo sobre lienzo,1632
42629,"Martirio del padre Andrés, prior de la Cartuja...","CARDUCHO, VICENTE",Desconocido,Desconocido,Desconocido,Desconocido,"Eslovenia, Seiz",Desconocido,Desconocido,Óleo sobre lienzo,1632
42631,"Martirio del padre Andrés, prior de la Cartuja...","CARDUCHO, VICENTE",Desconocido,Desconocido,Desconocido,Desconocido,"Eslovenia, Seiz",Desconocido,Desconocido,Óleo sobre lienzo,1632
42632,Lecture on Sculpture by Sir Richard...,"SCHARF EL VIEJO, GEORGE - LITÓGRAFO; HULLMANDE...",Desconocido,Desconocido,Desconocido,Desconocido,"Reino Unido, Londres",Desconocido,Desconocido,"Cromolitografía, Estampa iluminada sobre papel",Desconocido


In [41]:
cuadros_completo.duplicated().any()

True

In [42]:
cuadros_completo = cuadros_completo.drop_duplicates()

# Creación de TituloID

In [44]:

# Crear una nueva columna 'id'
cuadros_completo['TituloID'] = 0

# Inicializar el contador de id
contador_id = 1

# Iterar sobre las filas del DataFrame
for i, fila in cuadros_completo.iterrows():
    # Crear una condición para comparar las columnas deseadas
    condition = (cuadros_completo['Autor'] == fila['Autor']) & \
                (cuadros_completo['Titulo'] == fila['Titulo']) & \
                (cuadros_completo['Fecha'] == fila['Fecha']) & \
                (cuadros_completo['Escuela'] == fila['Escuela']) & \
                (cuadros_completo['Ubicacion'] == fila['Ubicacion'])
    
    # Asignar el mismo id a las filas que cumplan la condición
    cuadros_completo.loc[condition, 'TituloID'] = contador_id
    
    # Incrementar el contador de id
    contador_id += 1


In [48]:
cuadros_completo.tail()

,TituloID,Titulo,Autor,Fecha,Escuela,Objetos,Fauna,Flora,Geografico,Personajes
42613,15203,Isla de Cuba. La Ceiba,FREDRICKS Y DARIES - FOTÓGRAFO - (AUTOR DE LA ...,1885,Desconocido,Desconocido,Desconocido,Desconocido,"Antillas, Cuba",Desconocido
42618,15204,Recuperación de la isla de San Cristóbal,"JOLLIVET, PIERRE-JULES - LITÓGRAFO - (AUTOR DE...",Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,"Antillas, Isla De San Cristóbal",Desconocido
42625,15205,Incendio en la Cartuja de Praga y persecución ...,"CARDUCHO, VICENTE",1632,Desconocido,Desconocido,Desconocido,Desconocido,"República Checa (Chequia), Praga",Desconocido
42629,15206,"Martirio del padre Andrés, prior de la Cartuja...","CARDUCHO, VICENTE",1632,Desconocido,Desconocido,Desconocido,Desconocido,"Eslovenia, Seiz",Desconocido
42632,15207,Lecture on Sculpture by Sir Richard...,"SCHARF EL VIEJO, GEORGE - LITÓGRAFO; HULLMANDE...",Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,"Reino Unido, Londres",Desconocido


In [46]:
nuevo_orden = ["TituloID", "Titulo", "Autor", "Fecha", "Escuela", "Objetos", "Fauna", "Flora", "Geografico", "Personajes"]
cuadros_completo = cuadros_completo.reindex(columns=nuevo_orden)

In [47]:
cuadros_completo.head()

,TituloID,Titulo,Autor,Fecha,Escuela,Objetos,Fauna,Flora,Geografico,Personajes
0,1,El sumo sacerdote Aarón,"JUANES, JUAN DE",1550,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Aarón
1,3,Los israelitas bebiendo el agua milagrosa,"BASSANO, JACOPO",1568,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Aarón
2,3,Los israelitas bebiendo el agua milagrosa,"BASSANO, JACOPO",1568,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Moisés
3,20,Retablo de la Virgen,MAESTRO DE TORRALBA,1440,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Abacuc
4,20,Retablo de la Virgen,MAESTRO DE TORRALBA,1440,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Amós


# Exportación del DataFrame.

In [49]:
#cuadros_completo.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/cuadros_estudio.csv", index=False)